# Author: Seunghee Kim
- Created on: 2024-12-05

In [ ]:
# Notebook Explain
# 기존에 train / valid / test 데이터셋을 구축해서 deberta-v3-xsmall 모델로 train data 학습 후 test dataset에 대해 예측했는데, 정확도 0.99가 나오게 됨.
# 데이터셋의 난이도가 너무 쉽다는 문제점 발생
# 원인 분석
# 원인: train, valid, test 를 이루고 있는 AI-Generated Text가 모두 GPT-4o-mini 모델로 구성됨.
# 즉, GPT-4o-mini 모델로 만들어진 AI-Generated Text로 학습을 했으니 그 패턴을 파악하게 돼서 GPT-4-mini 모델로 만들어진 Test dataset이 난이도가 쉬워서 정확하게 분류가 가능하게 된 것
# 해결 방안: train & valid data와 test data의 distribution을 다르게 하기 위해서, test dataset은 GPT-4o-mini Model뿐만 아니라 최대한 다양한 모델을 이용해서 구축하게 되면 Custom dataset의 난이도가 더 향상될 것이다.

In [ ]:
# train dataset_v2: Human Text 4500개, AI-Text: GPT-4o-mini Text 4500개 총 9000개
# valid dataset_v2: Human Text 450개, AI-Text: GPT-4o-mini Text 450개 총 900개

# test dataset_v2: Human Text 450개, 
# AI-Text:
# 1.'gpt-4o' 50개
# 2.'meta-llama/Llama-3.2-8B-Instruct' 50개
# 3.'Qwen/Qwen2.5-7B-Instruct' 50개
# 4.'Qwen/Qwen2.5-14B-Instruct' 50개
# 5.'Qwen/Qwen2.5-1.5B-Instruct' 50개
# 6.'Qwen/Qwen2.5-0.5B-Instruct' 50개
# 7.'gpt-3.5-turbo' 50개 
# 8.'o1-mini' 50개 
# 9.'gpt-4o-mini' 50개 
# 총 900개

# Import Library

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
import yaml
from openai import OpenAI

# Config & Path

In [ ]:
class CFG:
    SEED = 1
    
    MODEL_1 = "gpt-4o" 
    MODEL_2 = 'meta-llama/Llama-3.1-8B-Instruct'
    MODEL_3 = 'Qwen/Qwen2.5-7B-Instruct'
    MODEL_4 = 'Qwen/Qwen2.5-14B-Instruct'
    MODEL_5 = 'Qwen/Qwen2.5-1.5B-Instruct'
    MODEL_6 = 'Qwen/Qwen2.5-0.5B-Instruct'
    MODEL_7 = 'gpt-3.5-turbo' 
    MODEL_8 = 'o1-mini' 
    MODEL_9 = 'gpt-4o-mini' 
    
    # api key가 담긴 yaml파일 (해당 yaml 파일은 .gitignore에 반드시 추가해야 함!!!)
    API_CONFIG_PATH = './config.yaml'
    
    # 1_dataset_preprocess.ipynb의 결과로 나온 전처리된 Human Dataset 경로
    DF_HUMAN_TEST_PATH = './df_human_test.csv'
    
    # AI-Generated Text까지 포함된 csv파일의 output 경로
    DF_HUMAN_AI_TEST_PATH_1 = './df_human_ai_test_1_gpt-4o.csv'
    DF_HUMAN_AI_TEST_PATH_2 = './df_human_ai_test_2_Llama-3_1-8B-Instruct.csv' 
    DF_HUMAN_AI_TEST_PATH_3 = './df_human_ai_test_3_Qwen2_5-7B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_4 = './df_human_ai_test_4_Qwen2_5-14B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_5 = './df_human_ai_test_5_Qwen2_5-1_5B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_6 = './df_human_ai_test_6_Qwen2_5-0_5B-Instruct.csv'
    DF_HUMAN_AI_TEST_PATH_7 = './df_human_ai_test_7_gpt-3_5_turbo.csv'
    DF_HUMAN_AI_TEST_PATH_8 = './df_human_ai_test_8_o1-mini.csv' 
    DF_HUMAN_AI_TEST_PATH_9 = './df_human_ai_test_9_gpt-4o-mini.csv'
    
    # Final Dataset output 경로 (Human Text, AI Text, Label 존재)
    DF_FINAL_TEST_PATH = './df_final_test_v2.csv'
    
    # CACHE_DIR = '/data/ksh1234/ksh1234/huggingface_cache' 


In [ ]:
# yaml파일에 존재하는 openai api key를 load 하는 함수
def load_api_key(yaml_file):
    with open(yaml_file, 'r', encoding='utf-8') as file:
        config = yaml.safe_load(file)
    return config['openai']['api_key_ksh']

api_key = load_api_key(CFG.API_CONFIG_PATH)
client = OpenAI(api_key=api_key)

In [ ]:
# SPLIT TEST DATASET INTO 50 * 9
test_dataset = pd.read_csv(CFG.DF_HUMAN_TEST_PATH)
split_datasets = []
chunk_size = 50

for i in range(0, len(test_dataset), chunk_size):
    split_datasets.append(test_dataset.iloc[i:i + chunk_size])

test_dataset_1 = split_datasets[0]
test_dataset_2 = split_datasets[1]
test_dataset_3 = split_datasets[2]
test_dataset_4 = split_datasets[3]
test_dataset_5 = split_datasets[4]
test_dataset_6 = split_datasets[5]
test_dataset_7 = split_datasets[6]
test_dataset_8 = split_datasets[7]
test_dataset_9 = split_datasets[8]

# Model 1. gpt-4o

In [ ]:
test_dataset_1['AI_Generated_Text'] = ""
test_dataset_1['Model'] = ""

for idx, row in tqdm(test_dataset_1.iterrows(), total=len(test_dataset_1)):
    input_text = row['essay_prompt']
    student_grade = row['student_grade']
    school, grade = student_grade.split('_')
    if school == '중등':
        school = school[0] # 중등인 경우 '중'만 가져옴
    
    # 학교 수준에 따라 다르게 설정하는 시스템 프롬프트
    grade_system_prompt = f"대한민국의 {school}학교 {grade} 학생 수준으로 답하시오. 마크다운 용법을 사용하지 말고 학생이 글을 쓰듯이 답하시오."
    response = client.chat.completions.create(
        model=CFG.MODEL_1,
        messages=[
            {"role": "system", "content": grade_system_prompt},
            {"role": "user", "content": input_text}
        ]
    )
    
    llm_output = response.choices[0].message.content
    # LLM이 생성한 Text를 'AI_Generated_Text' 열에 추가
    test_dataset_1.at[idx, 'AI_Generated_Text'] = llm_output
    test_dataset_1.at[idx, 'Model'] = CFG.MODEL_1

test_dataset_1.to_csv(CFG.DF_HUMAN_AI_TEST_PATH_1, index=False)
